In [ ]:
# this file should be used from the root of the repository
import pyeddl.eddl as eddl
import pyecvl.ecvl as ecvl
from pyeddl.tensor import Tensor

import pandas as pd
import numpy as np
from posixpath import join
import yaml
import os

from utils.data_partitioning import load_data_split
from eddl_lib.uc5_dataset import Uc5Dataset

# paths
exp_fld = "/mnt/datasets/uc5/UC5_pipeline_forked/experiments_eddl/wp6"
cnn_fn = "cnn_84val_neptune179.onnx"
ds_fn = "img_reports_phi2_enc.tsv"
img_fld = "/mnt/datasets/uc5/std-dataset/image"

# read files from exp_fld
train_ids, valid_ids, test_ids = load_data_split(exp_fld)

cnn = eddl.import_net_from_onnx_file(join(exp_fld, cnn_fn))
eddl.build(
    cnn,
    eddl.rmsprop(0.01),
    ["soft_cross_entropy"],
    ["categorical_accuracy"],
    eddl.CS_GPU(mem="full_mem"),  # if args.gpu else eddl.CS_CPU(mem=args.mem),
    False  # do not initialize weights to random values
)
cnn.resize(1)
# eddl.summary(cnn)
eddl.set_mode(cnn, 0)

ds = pd.read_csv(join(exp_fld, ds_fn), sep="\t").set_index("filename")  # .set_index("image_filename")
print(ds.shape)
print(ds.head())

semantic_dim = eddl.getLayer(cnn, "cnn_out").output.shape[1]
print("semantic dimension:", semantic_dim)


In [ ]:
# aux functions
def load_image(filename):
    augs = ecvl.SequentialAugmentationContainer([
                ecvl.AugToFloat32(divisor=255.0),
                ecvl.AugNormalize([0.48197903, 0.48197903, 0.48197903], [0.26261734, 0.26261734, 0.26261734]),
                ecvl.AugResizeDim([300, 300]),
                ecvl.AugCenterCrop([224, 224]),  # to do: test random crop also in prediction
                ])
    img = ecvl.ImRead(filename, flags=None)  # , flags=ecvl.ImReadMode.GRAYSCALE)
    ecvl.RearrangeChannels(img, img, "xyc")
    augs.Apply(img)
    ecvl.RearrangeChannels(img, img, "cxy")
    return img

def label_list(lab_str):
    return [int(s) for s in lab_str.split(";")]

In [ ]:
split_ids = train_ids + valid_ids
# split_ids = test_ids

predictions = np.empty( (semantic_dim, len(split_ids)) )
targets = np.empty_like(predictions)
predictions.fill(np.nan)
targets.fill(np.nan)

for pos, id in enumerate(split_ids):
    if (pos % 100) == 0:
        print(".", end="")
    img = load_image(join(img_fld, id))
    # img = ecvl.ImageToTensor(img)
    a = np.expand_dims(np.array(img, copy=False), axis=0)  # add batch dimension
    eddl.forward(cnn, [Tensor.fromarray(a)])

    layer = eddl.getLayer(cnn, "cnn_out")
    p = np.array(eddl.getOutput(layer), copy=False)
    predictions[:, pos] = p
    
    labels = np.zeros_like(p)
    lab_list = label_list(ds.loc[id].labels)
    # print(f"{pos}: n_labels: {len(lab_list)}")
    for l in lab_list:
        labels[0, l] = 1
    targets[:, pos] = labels
    
    # n_labels = nnz(labels)
    # if n_labels != len(lab_list):
    #     print(pos)
    #     print(lab_list)
    #     print(labels)
    
    # print(labels)
    # if pos == 50:
    #     print("dev mode, breaking at 50")
    #     break
    #print(p.shape)
    # cnn_out_in = eddl.Input([semantic_dim], name="in_semantic_features")
# for i, split in enumerate([train_ids, valid_ids, test_ids]):

print("cell done.")

In [ ]:
from sklearn.metrics import roc_curve, accuracy_score
import matplotlib.pyplot as plt
import math
from numpy import count_nonzero as nnz

auc_acc = []
J_acc = []

auc_ths = []
J_ths = []

for i in range(predictions.shape[0]):  # for i over labels
    y_est = predictions[i, :]  # these are the predictions made by the cnn
    y = targets[i, :]  # there are the true target values 
    fpr, tpr, thresholds = roc_curve(y, y_est)  # check threshoold
    
    # auc
    crit = np.sqrt(tpr * (1 - fpr) )
    m2 = thresholds[np.argmax(crit)]
    auc_ths.append(m2)
    y_est1 = np.where(y_est > m2, 1, 0)
    auc_acc.append(accuracy_score(y_est1, y) * 100)

    # Youden
    J = tpr - fpr
    ij = np.argmax(J)
    th_j = thresholds[ij]
    J_ths.append(th_j)
    y_est4 = np.where(y_est > th_j, 1, 0)
    J_acc.append(accuracy_score(y_est4, y)*100)

    print(f"*** label {i}")
    print(f"  - auc1:", auc_acc[-1])
    print(f"  - Youden:", J_acc[-1])
    print(f"auc {m2}, youden {th_j}")

In [ ]:
from sklearn.metrics import roc_curve, accuracy_score
import matplotlib.pyplot as plt
import math
from numpy import count_nonzero as nnz

auc_acc = []
J_acc = []

auc_ths = []
J_ths = []

for i in range(predictions.shape[0]):  # for i over labels
    y_est = predictions[i, :]  # these are the predictions made by the cnn
    y = targets[i, :]  # there are the true target values 
    fpr, tpr, thresholds = roc_curve(y, y_est)  # check threshoold
    
    # auc
    crit = np.sqrt(tpr * (1 - fpr) )
    m2 = thresholds[np.argmax(crit)]
    auc_ths.append(m2)
    y_est1 = np.where(y_est > m2, 1, 0)
    auc_acc.append(accuracy_score(y_est1, y) * 100)

    # Youden
    J = tpr - fpr
    ij = np.argmax(J)
    th_j = thresholds[ij]
    J_ths.append(th_j)
    y_est4 = np.where(y_est > th_j, 1, 0)
    J_acc.append(accuracy_score(y_est4, y)*100)

    print(f"*** label {i}")
    print(f"  - auc1:", auc_acc[-1])
    print(f"  - Youden:", J_acc[-1])
    print(f"auc {m2}, youden {th_j}")

In [ ]:
d = {"auc_t": auc_ths, "auc_acc": auc_acc, "youden_t": J_ths, "youden_acc": J_acc}
df = pd.DataFrame.from_dict(d)
df["different"] = df.auc_t != df.youden_t
display(df)
print(df[["auc_acc", "youden_acc"]].mean(axis=0))

fn = join(exp_fld, "thresholds_trva.tsv")
# fn = join(exp_fld, "thresholds_te.tsv")
df.to_csv(fn, sep="\t" ) 
print("saved:", fn)


In [ ]:
values = [[10, 8.1, 4, 5], [1, 3, 80, 20], [5,9,78,1]]
a = np.array(values)
th = [9, 7, 77, 8]
t = np.array(th)

print(a.shape)
print(a)
print(t)
b = np.where(a < th, 0, 1)
print(b)